<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Módulo** | Análise de Dados: COVID-19 Dashboard
Caderno de **Exercícios**<br>
Professor [André Perez](https://www.linkedin.com/in/andremarcosperez/)

---

# **Tópicos**

<ol type="1">
  <li>Introdução;</li>
  <li>Análise Exploratória de Dados;</li>
  <li>Visualização Interativa de Dados;</li>
  <li>Storytelling.</li>
</ol>

---

# **Exercícios**

Este *notebook* deve servir como um guia para **você continuar** a construção da sua própria análise exploratória de dados interativa. Fique a vontate para copiar os códigos da aula mas busque explorar os dados ao máximo. Por fim, publique seu *notebook* no [Kaggle](https://www.kaggle.com/) e seu *dashboard* [Google Data Studio](https://datastudio.google.com/).

---

## 1\. Introdução

### **1.1. TLDR**

In [ ]:
 - **Dashboard**:
  - Google Data Studio (`https://lookerstudio.google.com/reporting/6a4a5638-0e93-4266-8022-03ecaa9d7612`).
 - **Processamento**:
  - Kaggle Notebook (`https://www.kaggle.com/code/mateus4422/notebook-covid`).
 - **Fontes**:
  - Casos pela universidade John Hopkins ([link](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports));
  - Vacinação pela universidade de Oxford ([link](https://covid.ourworldindata.org/data/owid-covid-data.csv)).

### **1.2. Pandemia Coronavírus 2019**

> A COVID-19 é uma infecção respiratória aguda causada pelo coronavírus SARS-CoV-2, potencialmente grave, de elevada transmissibilidade e de distribuição global. Fonte: Governo brasileiro ([link](https://www.gov.br/saude/pt-br/coronavirus/o-que-e-o-coronavirus)).

A disponibilidade de dados sobre a evolução da pandemia no tempo em uma determinada região geográfica é fundamental para o seu combate! Este projeto busca construir um dashboard de dados para exploração e visualização interativa de dados sobre o avanço de casos e da vacinação do Brasil. O processamento de dados está neste `link` e o dashboard, neste `link`.

### **1.3. Dados**

Os dados sobre **casos da COVID-19** são compilados pelo centro de ciência de sistemas e engenharia da universidade americana **John Hopkins** ([link](https://www.jhu.edu)). Os dados são atualizados diariamente deste janeiro de 2020 com uma granularidade temporal de dias e geográfica de regiões de países (estados, condados, etc.). O website do projeto pode ser acessado neste [link](https://systems.jhu.edu/research/public-health/ncov/) enquanto os dados, neste [link](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports). Abaixo estão descritos os dados derivados do seu processamento.

 - **date**: data de referência;
 - **state**: estado;
 - **country**: país;
 - **population**: população estimada;
 - **confirmed**: número acumulado de infectados;
 - **confirmed_1d**: número diário de infectados;
 - **confirmed_moving_avg_7d**: média móvel de 7 dias do número diário de infectados;
 - **confirmed_moving_avg_7d_rate_14d**: média móvel de 7 dias dividido pela média móvel de 7 dias de 14 dias atrás;
 - **deaths**: número acumulado de mortos;
 - **deaths_1d**: número diário de mortos;
 - **deaths_moving_avg_7d**: média móvel de 7 dias do número diário de mortos;
 - **deaths_moving_avg_7d**: média móvel de 7 dias dividido pela média móvel de 7 dias de 14 dias atrás;
 - **month**: mês de referência;
 - **year**: ano de referência.

Os dados sobre **vacinação da COVID-19** são compilados pelo projeto Nosso Mundo em Dados (*Our World in Data* ou OWID) da universidade britânica de **Oxford** ([link](https://www.ox.ac.uk)). Os dados são **atualizados diariamente** deste janeiro de 2020 com uma **granularidade temporal de dias e geográfica de países**. O website do projeto pode ser acessado neste [link](https://ourworldindata.org) enquanto os dados, neste [link](https://covid.ourworldindata.org/data/owid-covid-data.csv). Abaixo estão descritos os dados derivados do seu processamento.

 - **date**: data de referência;
 - **country**: país;
 - **population**: população estimada;
 - **total**: número acumulado de doses administradas;
 - **one_shot**: número acumulado de pessoas com uma dose;
 - **one_shot_perc**: número acumulado relativo de pessoas com uma dose;
 - **two_shots**: número acumulado de pessoas com duas doses;
 - **two_shot_perc**: número acumulado relativo de pessoas com duas doses;
 - **three_shots**: número acumulado de pessoas com três doses;
 - **three_shot_perc**: número acumulado relativo de pessoas com três doses;
 - **month**: mês de referência;
 - **year**: ano de referência.

## 2\. Análise Exploratória de Dados

Nesta sessão vamos utilizar os seguintes pacotes Python para processar os dados bruto em um formato adequado para um painel para exploração interativa de dados.

In [1]:
import math
from typing import Iterator
from datetime import datetime, timedelta

import numpy as np
import pandas as pd

### **2.1. Casos**

Vamos processar os dados de **casos** da universidade John Hopkins.


#### **2.1.1. Extração**

O dado está compilado em um arquivo por dia, exemplo para 2021/12/01.

In [2]:
cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-12-2021.csv', sep=',')

In [3]:
cases.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-01-13 05:22:15,33.93911,67.709953,53584,2301,44608,6675,Afghanistan,137.647787,4.294192
1,NaN,NaN,NaN,Albania,2021-01-13 05:22:15,41.15330,20.168300,64627,1252,38421,24954,Albania,2245.708527,1.937271
2,NaN,NaN,NaN,Algeria,2021-01-13 05:22:15,28.03390,1.659600,102641,2816,69608,30217,Algeria,234.067409,2.743543
3,NaN,NaN,NaN,Andorra,2021-01-13 05:22:15,42.50630,1.521800,8682,86,7930,666,Andorra,11236.653077,0.990555
4,NaN,NaN,NaN,Angola,2021-01-13 05:22:15,-11.20270,17.873900,18343,422,15512,2409,Angola,55.811022,2.300605


Portanto, precisaremos iterar dentro de um intervalo de tempo definido para extraí-lo.

In [4]:
def date_range(start_date: datetime, end_date: datetime) -> Iterator[datetime]:
  date_range_days: int = (end_date - start_date).days
  for lag in range(date_range_days):
    yield start_date + timedelta(lag)

In [5]:
start_date = datetime(2021,  1,  1)
end_date   = datetime(2021, 12, 31)

De maneira iterativa, vamos selecionar as colunas de interesse e as linhas referentes ao Brasil.

In [6]:
cases = None
cases_is_empty = True

for date in date_range(start_date=start_date, end_date=end_date):

  date_str = date.strftime('%m-%d-%Y')
  data_source_url = f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{date_str}.csv'

  case = pd.read_csv(data_source_url, sep=',')

  case = case.drop(['FIPS', 'Admin2', 'Last_Update', 'Lat', 'Long_', 'Recovered', 'Active', 'Combined_Key', 'Case_Fatality_Ratio'], axis=1)
  case = case.query('Country_Region == "Brazil"').reset_index(drop=True)
  case['Date'] = pd.to_datetime(date.strftime('%Y-%m-%d'))
  if cases_is_empty:
    cases = case
    cases_is_empty = False
  else:
    cases = pd.concat([cases, case], axis= 0, ignore_index=True)

In [7]:
cases.query('Province_State == "Sao Paulo"').head()

,Province_State,Country_Region,Confirmed,Deaths,Incident_Rate,Date
24,Sao Paulo,Brazil,1466191,46775,3192.990778,2021-01-01
51,Sao Paulo,Brazil,1467953,46808,3196.827966,2021-01-02
78,Sao Paulo,Brazil,1471422,46845,3204.382565,2021-01-03
105,Sao Paulo,Brazil,1473670,46888,3209.278136,2021-01-04
132,Sao Paulo,Brazil,1486551,47222,3237.329676,2021-01-05


#### **2.1.2. Wrangling**

Vamos manipular os dados para o dashboard. O foco é em garantir uma boa granularidade e qualidade da base de dados.

In [8]:
cases.head()

,Province_State,Country_Region,Confirmed,Deaths,Incident_Rate,Date
0,Acre,Brazil,41689,796,4726.992352,2021-01-01
1,Alagoas,Brazil,105091,2496,3148.928928,2021-01-01
2,Amapa,Brazil,68361,926,8083.066602,2021-01-01
3,Amazonas,Brazil,201574,5295,4863.536793,2021-01-01
4,Bahia,Brazil,494684,9159,3326.039611,2021-01-01


In [9]:
cases.shape

(9828, 6)

In [10]:
cases.info()

<class 'pandas.core.frame.DataFrame'>RangeIndex: 9828 entries, 0 to 9827Data columns (total 6 columns): #   Column          Non-Null Count  Dtype         ---  ------          --------------  -----          0   Province_State  9828 non-null   object         1   Country_Region  9828 non-null   object         2   Confirmed       9828 non-null   int64          3   Deaths          9828 non-null   int64          4   Incident_Rate   9828 non-null   float64        5   Date            9828 non-null   datetime64[ns]dtypes: datetime64[ns](1), float64(1), int64(2), object(2)memory usage: 460.8+ KB

Começamos com o nome das colunas.

In [11]:
cases = cases.rename(
  columns={
    'Province_State': 'state',
    'Country_Region': 'country'
  }
)

for col in cases.columns:
  cases = cases.rename(columns={col: col.lower()})

Ajustamos o nome dos estados.

In [12]:
states_map = {
    'Amapa': 'Amapá',
    'Ceara': 'Ceará',
    'Espirito Santo': 'Espírito Santo',
    'Goias': 'Goiás',
    'Para': 'Pará',
    'Paraiba': 'Paraíba',
    'Parana': 'Paraná',
    'Piaui': 'Piauí',
    'Rondonia': 'Rondônia',
    'Sao Paulo': 'São Paulo',
    'Maranhao': 'Maranhão'
}

cases['state'] = cases['state'].apply(lambda state: states_map.get(state) if state in states_map.keys() else state)

Vamos então computar novas colunas para enriquecer a base de dados.

Vamos então computar novas colunas para enriquecer a base de dados.

 - População estimada do estado:

In [13]:
cases['population'] = round(100000 * (cases['confirmed'] / cases['incident_rate']))
cases = cases.drop('incident_rate', axis=1)

 - Chaves temporais:

In [14]:
cases['month'] = cases['date'].apply(lambda date: date.strftime('%Y-%m'))
cases['year']  = cases['date'].apply(lambda date: date.strftime('%Y'))

 - Número, média móvel (7 dias) e estabilidade (14 dias) de casos e mortes por estado:

| 1 | 2 | 3 | 4 | 5 | 6 | <font color='red'>7</font> | <font color='green'>8</font> | 9 | 10 | 11 | 12 | 13 | <font color='blue'>14<font color='red'> | 15 | 16 | 17 | 18 | 19 | 20 | 21 |
| - | - | - | - | - | - | - | - | - | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- |
| <font color='red'>D-6</font> | <font color='red'>D-5</font> | <font color='red'>D-4</font> | <font color='red'>D-3</font> | <font color='red'>D-2</font> | <font color='red'>D-1</font> | <font color='red'>D0</font> | | | | | | | | | | | | | | |
| D-7 | <font color='green'>D-6</font> | <font color='green'>D-5</font> | <font color='green'>D-4</font> | <font color='green'>D-3</font> | <font color='green'>D-2</font> | <font color='green'>D-1</font> | <font color='green'>D0</font> | | | | | | | | | | | | | |
| D-13 | D-12 | D-11 | D-10 | D-9 | D-8 | D-7 | <font color='blue'>D-6</font> | <font color='blue'>D-5</font> | <font color='blue'>D-4</font> | <font color='blue'>D-3</font> | <font color='blue'>D-2</font> | <font color='blue'>D-1</font> | <font color='blue'>D0</font> | | | | | | | |

In [15]:
cases_ = None
cases_is_empty = True

def get_trend(rate: float) -> str:

  if np.isnan(rate):
    return np.NaN

  if rate < 0.85:
    status = 'downward'
  elif rate > 1.15:
    status = 'upward'
  else:
    status = 'stable'

  return status


for state in cases['state'].drop_duplicates():

  cases_per_state = cases.query(f'state == "{state}"').reset_index(drop=True)
  cases_per_state = cases_per_state.sort_values(by=['date'])

  cases_per_state['confirmed_1d'] = cases_per_state['confirmed'].diff(periods=1)
  cases_per_state['confirmed_moving_avg_7d'] = np.ceil(cases_per_state['confirmed_1d'].rolling(window=7).mean())
  cases_per_state['confirmed_moving_avg_7d_rate_14d'] = cases_per_state['confirmed_moving_avg_7d']/cases_per_state['confirmed_moving_avg_7d'].shift(periods=14)
  cases_per_state['confirmed_trend'] = cases_per_state['confirmed_moving_avg_7d_rate_14d'].apply(get_trend)

  cases_per_state['deaths_1d'] = cases_per_state['deaths'].diff(periods=1)
  cases_per_state['deaths_moving_avg_7d'] = np.ceil(cases_per_state['deaths_1d'].rolling(window=7).mean())
  cases_per_state['deaths_moving_avg_7d_rate_14d'] = cases_per_state['deaths_moving_avg_7d']/cases_per_state['deaths_moving_avg_7d'].shift(periods=14)
  cases_per_state['deaths_trend'] = cases_per_state['deaths_moving_avg_7d_rate_14d'].apply(get_trend)

  if cases_is_empty:
    cases_ = cases_per_state
    cases_is_empty = False
  else:
    cases_ = pd.concat([cases_, cases_per_state],axis=0, ignore_index=True)

cases = cases_
cases_ = None

Garantir o tipo do dado é fundamental para consistência da base de dados. Vamos fazer o *type casting* das colunas.

In [16]:
cases['population'] = cases['population'].astype('Int64')
cases['confirmed_1d'] = cases['confirmed_1d'].astype('Int64')
cases['confirmed_moving_avg_7d'] = cases['confirmed_moving_avg_7d'].astype('Int64')
cases['deaths_1d'] = cases['deaths_1d'].astype('Int64')
cases['deaths_moving_avg_7d'] = cases['deaths_moving_avg_7d'].astype('Int64')

Vamos reorganizar as colunas.

In [17]:
cases = cases[['date', 'country', 'state', 'population', 'confirmed', 'confirmed_1d', 'confirmed_moving_avg_7d', 'confirmed_moving_avg_7d_rate_14d', 'confirmed_trend', 'deaths', 'deaths_1d', 'deaths_moving_avg_7d', 'deaths_moving_avg_7d_rate_14d', 'deaths_trend','month','year']]
cases.head(15)

,date,country,state,population,confirmed,confirmed_1d,confirmed_moving_avg_7d,confirmed_moving_avg_7d_rate_14d,confirmed_trend,deaths,deaths_1d,deaths_moving_avg_7d,deaths_moving_avg_7d_rate_14d,deaths_trend,month,year
0,2021-01-01,Brazil,Acre,881935,41689,<NA>,<NA>,NaN,NaN,796,<NA>,<NA>,NaN,NaN,2021-01,2021
1,2021-01-02,Brazil,Acre,881935,41941,252,<NA>,NaN,NaN,798,2,<NA>,NaN,NaN,2021-01,2021
2,2021-01-03,Brazil,Acre,881935,42046,105,<NA>,NaN,NaN,802,4,<NA>,NaN,NaN,2021-01,2021
3,2021-01-04,Brazil,Acre,881935,42117,71,<NA>,NaN,NaN,806,4,<NA>,NaN,NaN,2021-01,2021
4,2021-01-05,Brazil,Acre,881935,42170,53,<NA>,NaN,NaN,808,2,<NA>,NaN,NaN,2021-01,2021
5,2021-01-06,Brazil,Acre,881935,42378,208,<NA>,NaN,NaN,814,6,<NA>,NaN,NaN,2021-01,2021
6,2021-01-07,Brazil,Acre,881935,42478,100,<NA>,NaN,NaN,821,7,<NA>,NaN,NaN,2021-01,2021
7,2021-01-08,Brazil,Acre,881935,42814,336,161,NaN,NaN,823,2,4,NaN,NaN,2021-01,2021
8,2021-01-09,Brazil,Acre,881935,42908,94,139,NaN,NaN,823,0,4,NaN,NaN,2021-01,2021
9,2021-01-10,Brazil,Acre,881935,43127,219,155,NaN,NaN,825,2,4,NaN,NaN,2021-01,2021


In [18]:
#tranformando mes em numero do mes
cases['month'] = cases['date'].dt.month
cases.head(1)

,date,country,state,population,confirmed,confirmed_1d,confirmed_moving_avg_7d,confirmed_moving_avg_7d_rate_14d,confirmed_trend,deaths,deaths_1d,deaths_moving_avg_7d,deaths_moving_avg_7d_rate_14d,deaths_trend,month,year
0,2021-01-01,Brazil,Acre,881935,41689,<NA>,<NA>,NaN,NaN,796,<NA>,<NA>,NaN,NaN,1,2021


Por fim vamos traduzir o nome do DataFrame e das colunas e depois conferir o resultado final.

In [19]:
casos = cases.copy()

In [20]:
casos.columns = ['data', 'país', 'estado', 'populacao', 'acumulado_infectados', 'infectados_diario', 'media_movel_7d_infectados', 'media_movel_7d_infectados/media_movel_7d_de_14d_atrás', 'tendencia_infectados', 'acumulado_mortes', 'mortes_diarias', 'media_movel_7d_mortes', 'media_movel_7d_mortos/media_movel_7d_de_14d_atras', 'tendencia_mortes','mes', 'ano']

In [21]:
casos.tail(n=5)

,data,país,estado,populacao,acumulado_infectados,infectados_diario,media_movel_7d_infectados,media_movel_7d_infectados/media_movel_7d_de_14d_atrás,tendencia_infectados,acumulado_mortes,mortes_diarias,media_movel_7d_mortes,media_movel_7d_mortos/media_movel_7d_de_14d_atras,tendencia_mortes,mes,ano
9823,2021-12-26,Brazil,Tocantins,1572866,234113,0,0,0.000000,downward,3927,0,0,0.0,downward,12,2021
9824,2021-12-27,Brazil,Tocantins,1572866,234113,0,0,0.000000,downward,3927,0,0,0.0,downward,12,2021
9825,2021-12-28,Brazil,Tocantins,1572866,234964,851,122,2.837209,upward,3933,6,1,1.0,stable,12,2021
9826,2021-12-29,Brazil,Tocantins,1572866,235340,376,176,inf,upward,3936,3,2,inf,upward,12,2021
9827,2021-12-30,Brazil,Tocantins,1572866,235558,218,207,inf,upward,3939,3,2,inf,upward,12,2021


#### **2.1.3. Carregamento**

Com os dados manipulados, vamos persisti-lo em disco, fazer o seu download e carrega-lo no Google Data Studio.

In [22]:
casos.to_csv('./casos-covid2.csv', sep=',', index=False)

#### **2.1.4. Métricas**

irei fazer mais um DataFrame , com base no Dataframe cases , para não deixa-lo muito cheio pois irei fazer medidas para o looker studio

In [23]:
casos_mensais=cases.copy()

In [24]:
casos_mensais.head(15)

,date,country,state,population,confirmed,confirmed_1d,confirmed_moving_avg_7d,confirmed_moving_avg_7d_rate_14d,confirmed_trend,deaths,deaths_1d,deaths_moving_avg_7d,deaths_moving_avg_7d_rate_14d,deaths_trend,month,year
0,2021-01-01,Brazil,Acre,881935,41689,<NA>,<NA>,NaN,NaN,796,<NA>,<NA>,NaN,NaN,1,2021
1,2021-01-02,Brazil,Acre,881935,41941,252,<NA>,NaN,NaN,798,2,<NA>,NaN,NaN,1,2021
2,2021-01-03,Brazil,Acre,881935,42046,105,<NA>,NaN,NaN,802,4,<NA>,NaN,NaN,1,2021
3,2021-01-04,Brazil,Acre,881935,42117,71,<NA>,NaN,NaN,806,4,<NA>,NaN,NaN,1,2021
4,2021-01-05,Brazil,Acre,881935,42170,53,<NA>,NaN,NaN,808,2,<NA>,NaN,NaN,1,2021
5,2021-01-06,Brazil,Acre,881935,42378,208,<NA>,NaN,NaN,814,6,<NA>,NaN,NaN,1,2021
6,2021-01-07,Brazil,Acre,881935,42478,100,<NA>,NaN,NaN,821,7,<NA>,NaN,NaN,1,2021
7,2021-01-08,Brazil,Acre,881935,42814,336,161,NaN,NaN,823,2,4,NaN,NaN,1,2021
8,2021-01-09,Brazil,Acre,881935,42908,94,139,NaN,NaN,823,0,4,NaN,NaN,1,2021
9,2021-01-10,Brazil,Acre,881935,43127,219,155,NaN,NaN,825,2,4,NaN,NaN,1,2021


In [25]:
#escolhendo variaveis para trabalhar
casos_mensais=casos_mensais[['date', 'country', 'state', 'population', 'confirmed', 'confirmed_1d', 'deaths', 'deaths_1d','month','year' ]]

In [26]:
casos_mensais.head(3)

,date,country,state,population,confirmed,confirmed_1d,deaths,deaths_1d,month,year
0,2021-01-01,Brazil,Acre,881935,41689,<NA>,796,<NA>,1,2021
1,2021-01-02,Brazil,Acre,881935,41941,252,798,2,1,2021
2,2021-01-03,Brazil,Acre,881935,42046,105,802,4,1,2021


In [27]:
#tranformando mes em numero do mes
casos_mensais['month'] = casos_mensais['date'].dt.month
casos_mensais.head(1)

<ipython-input-27-f0ec0d87cd4e>:2: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.Try using .loc[row_indexer,col_indexer] = value insteadSee the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy  casos_mensais['month'] = casos_mensais['date'].dt.month

,date,country,state,population,confirmed,confirmed_1d,deaths,deaths_1d,month,year
0,2021-01-01,Brazil,Acre,881935,41689,<NA>,796,<NA>,1,2021


 - Mês com mais casos, total de casos por mês e casos acumalados por mes :

In [28]:
# Agrupando por mês e somando os casos
casos_por_mes = casos_mensais.groupby('month')['confirmed_1d'].sum()

In [29]:
# Identificando o mês com mais casos
mes_mais_casos = casos_por_mes.idxmax()  # Retorna o número do mês com mais casos
casos_no_mes = casos_por_mes.max()  # Número total de casos no mês

In [30]:
# Adicionando uma nova coluna indicando o mês com mais casos
casos_mensais['mes_com_mais_casos'] = mes_mais_casos

In [31]:
# Criando a coluna 'total_mes' com o total de casos do mês atual
casos_mensais['total_casos_mes'] = casos_mensais['month'].map(casos_por_mes)
casos_mensais.head(2)

,date,country,state,population,confirmed,confirmed_1d,deaths,deaths_1d,month,year,mes_com_mais_casos,total_casos_mes
0,2021-01-01,Brazil,Acre,881935,41689,<NA>,796,<NA>,1,2021,3,1504550
1,2021-01-02,Brazil,Acre,881935,41941,252,798,2,1,2021,3,1504550


In [32]:
# Calcular o acumulado dos meses
casos_por_mes_acumulado = casos_por_mes.cumsum()
#Mapear o acumulado para o DataFrame
casos_mensais['casos_acumulado_mensal'] = casos_mensais['month'].map(casos_por_mes_acumulado)
casos_mensais.head(3)

,date,country,state,population,confirmed,confirmed_1d,deaths,deaths_1d,month,year,mes_com_mais_casos,total_casos_mes,casos_acumulado_mensal
0,2021-01-01,Brazil,Acre,881935,41689,<NA>,796,<NA>,1,2021,3,1504550,1504550
1,2021-01-02,Brazil,Acre,881935,41941,252,798,2,1,2021,3,1504550,1504550
2,2021-01-03,Brazil,Acre,881935,42046,105,802,4,1,2021,3,1504550,1504550


In [33]:
#conferindo as colunas novas separado do df
casos_mensais[['total_casos_mes','casos_acumulado_mensal', 'month', 'mes_com_mais_casos']].drop_duplicates()

,total_casos_mes,casos_acumulado_mensal,month,mes_com_mais_casos
0,1504550,1504550,1,3
31,1349570,2854120,2,3
59,2205301,5059421,3,3
90,1911712,6971133,4,3
120,1882784,8853917,5,3
151,2012408,10866325,6,3
181,1351850,12218175,7,3
212,863050,13081225,8,3
243,647166,13728391,9,3
273,382263,14110654,10,3


 - Mês com mais mortes, total de mortes por mês e mortes acumalados por mes :

In [34]:
#faremos o mesmo processo que fizemos com as colunas de casos
# Agrupando por mês e somando as mortes
mortes_por_mes = casos_mensais.groupby('month')['deaths_1d'].sum()

In [35]:
# Identificando o mês com mais mortes
mes_mais_mortes = mortes_por_mes.idxmax()  # Retorna o número do mês com mais mortes
mortes_no_mes = mortes_por_mes.max()  # Número total de mortes no mês

In [36]:
# Adicionando uma nova coluna indicando o mês com mais mortes
casos_mensais['mes_com_mais_mortes'] = mes_mais_mortes

In [37]:
# Criando a coluna 'total_mortes_por_mes' com o total de mortes do mês atual
casos_mensais['total_mortes_por_mes'] = casos_mensais['month'].map(mortes_por_mes)

In [38]:
#Calcular o acumulado dos meses
mortes_por_mes_acumulado = mortes_por_mes.cumsum()
#Mapear o acumulado para o DataFrame
casos_mensais['mortes_acumulado_mensal'] = casos_mensais['month'].map(mortes_por_mes_acumulado)
casos_mensais.head(3)

,date,country,state,population,confirmed,confirmed_1d,deaths,deaths_1d,month,year,mes_com_mais_casos,total_casos_mes,casos_acumulado_mensal,mes_com_mais_mortes,total_mortes_por_mes,mortes_acumulado_mensal
0,2021-01-01,Brazil,Acre,881935,41689,<NA>,796,<NA>,1,2021,3,1504550,1504550,4,29107,29107
1,2021-01-02,Brazil,Acre,881935,41941,252,798,2,1,2021,3,1504550,1504550,4,29107,29107
2,2021-01-03,Brazil,Acre,881935,42046,105,802,4,1,2021,3,1504550,1504550,4,29107,29107


In [39]:
#conferindo as colunas novas separado do df
casos_mensais[['total_mortes_por_mes','mortes_acumulado_mensal','mes_com_mais_mortes', 'month']].drop_duplicates()

,total_mortes_por_mes,mortes_acumulado_mensal,mes_com_mais_mortes,month
0,29107,29107,4,1
31,30531,59638,4,2
59,66959,126597,4,3
90,82392,208989,4,4
120,58714,267703,4,5
151,55244,322947,4,6
181,38188,361135,4,7
212,24088,385223,4,8
243,16268,401491,4,9
273,11050,412541,4,10


In [41]:
#juntarei o mes com mais casos com os mais casos desse mes em uma
#coluna para fazer um cartão no looker(tambem farei para as mortes)
meses = {
    1: "Janeiro", 2: "Fevereiro", 3: "Março", 4: "Abril",
    5: "Maio", 6: "Junho", 7: "Julho", 8: "Agosto",
    9: "Setembro", 10: "Outubro", 11: "Novembro", 12: "Dezembro"
}

casos_mensais['nome_do_mes'] = casos_mensais['month'].map(meses)
casos_mensais['nome_do_mes'] = casos_mensais['month'].map(meses)
casos_mensais['nome_mes_mais_casos']=casos_mensais['mes_com_mais_casos'].map(meses)
casos_mensais['nome_mes_mais_mortes']=casos_mensais['mes_com_mais_mortes'].map(meses)

In [42]:
# Encontra a quantidade de casos no mês com mais casos
total_casos_mes_com_mais_casos = casos_mensais['total_casos_mes'].max()
# Cria a coluna 'mes_mais_caso' com o nome do mês e a quantidade de casos
casos_mensais['mes_mais_caso'] = casos_mensais['nome_mes_mais_casos'] + '-' + str(total_casos_mes_com_mais_casos)

# Encontra a quantidade de mortes no mês com mais mortes
total_mortes_mes_com_mais_mortes = casos_mensais['total_mortes_por_mes'].max()
# Cria a coluna 'mes_mais_mortes' com o nome do mês e a quantidade de mortes
casos_mensais['mes_mais_mortes'] = casos_mensais['nome_mes_mais_mortes'] + '-' + str(total_mortes_mes_com_mais_mortes)
casos_mensais.head(3)

,date,country,state,population,confirmed,confirmed_1d,deaths,deaths_1d,month,year,...,total_casos_mes,casos_acumulado_mensal,mes_com_mais_mortes,total_mortes_por_mes,mortes_acumulado_mensal,nome_do_mes,nome_mes_mais_casos,nome_mes_mais_mortes,mes_mais_caso,mes_mais_mortes
0,2021-01-01,Brazil,Acre,881935,41689,<NA>,796,<NA>,1,2021,...,1504550,1504550,4,29107,29107,Janeiro,Março,Abril,Março-2205301,Abril-82392
1,2021-01-02,Brazil,Acre,881935,41941,252,798,2,1,2021,...,1504550,1504550,4,29107,29107,Janeiro,Março,Abril,Março-2205301,Abril-82392
2,2021-01-03,Brazil,Acre,881935,42046,105,802,4,1,2021,...,1504550,1504550,4,29107,29107,Janeiro,Março,Abril,Março-2205301,Abril-82392


In [43]:
casos_mensais.info()

<class 'pandas.core.frame.DataFrame'>RangeIndex: 9828 entries, 0 to 9827Data columns (total 21 columns): #   Column                   Non-Null Count  Dtype         ---  ------                   --------------  -----          0   date                     9828 non-null   datetime64[ns] 1   country                  9828 non-null   object         2   state                    9828 non-null   object         3   population               9828 non-null   Int64          4   confirmed                9828 non-null   int64          5   confirmed_1d             9801 non-null   Int64          6   deaths                   9828 non-null   int64          7   deaths_1d                9801 non-null   Int64          8   month                    9828 non-null   int32          9   year                     9828 non-null   object         10  mes_com_mais_casos       9828 non-null   int32          11  total_casos_mes          9828 non-null   Int64          12  casos_acumulado_mensal   9828 non-null   Int64     

 - Casos e Mortes por região

In [44]:
# criaremos um dicionario para criarmos a coluna de região
estados_para_regioes = {
    'Acre': 'Norte', 'Alagoas': 'Nordeste', 'Amapá': 'Norte', 'Amazonas': 'Norte',
    'Bahia': 'Nordeste', 'Ceará': 'Nordeste', 'Distrito Federal': 'Centro-Oeste', 'Espírito Santo': 'Sudeste',
    'Goiás': 'Centro-Oeste', 'Maranhão': 'Nordeste', 'Mato Grosso': 'Centro-Oeste',
    'Mato Grosso do Sul': 'Centro-Oeste', 'Minas Gerais': 'Sudeste', 'Pará': 'Norte', 'Paraíba': 'Nordeste',
    'Paraná': 'Sul', 'Pernambuco': 'Nordeste', 'Piauí': 'Nordeste', 'Rio de Janeiro': 'Sudeste',
    'Rio Grande do Norte': 'Nordeste', 'Rio Grande do Sul': 'Sul', 'Rondônia': 'Norte', 'Roraima': 'Norte',
    'Santa Catarina': 'Sul', 'São Paulo': 'Sudeste', 'Sergipe': 'Nordeste', 'Tocantins': 'Norte'
}

In [45]:
casos_mensais['regiao'] = casos_mensais['state'].map(estados_para_regioes)
casos_mensais.head(3)

,date,country,state,population,confirmed,confirmed_1d,deaths,deaths_1d,month,year,...,casos_acumulado_mensal,mes_com_mais_mortes,total_mortes_por_mes,mortes_acumulado_mensal,nome_do_mes,nome_mes_mais_casos,nome_mes_mais_mortes,mes_mais_caso,mes_mais_mortes,regiao
0,2021-01-01,Brazil,Acre,881935,41689,<NA>,796,<NA>,1,2021,...,1504550,4,29107,29107,Janeiro,Março,Abril,Março-2205301,Abril-82392,Norte
1,2021-01-02,Brazil,Acre,881935,41941,252,798,2,1,2021,...,1504550,4,29107,29107,Janeiro,Março,Abril,Março-2205301,Abril-82392,Norte
2,2021-01-03,Brazil,Acre,881935,42046,105,802,4,1,2021,...,1504550,4,29107,29107,Janeiro,Março,Abril,Março-2205301,Abril-82392,Norte


In [46]:
# aqui estamos agrupando a partir do mês e região e somando os infectados
#a granuralidade sera mensal , para vermos como foi a evolução nas regiões com o  passar dos meses
casos_mensais['casos_regiao_mensal'] = casos_mensais.groupby(['regiao', 'month'])['confirmed_1d'].transform('sum')
casos_mensais.tail()

,date,country,state,population,confirmed,confirmed_1d,deaths,deaths_1d,month,year,...,mes_com_mais_mortes,total_mortes_por_mes,mortes_acumulado_mensal,nome_do_mes,nome_mes_mais_casos,nome_mes_mais_mortes,mes_mais_caso,mes_mais_mortes,regiao,casos_regiao_mensal
9823,2021-12-26,Brazil,Tocantins,1572866,234113,0,3927,0,12,2021,...,4,4264,423708,Dezembro,Março,Abril,Março-2205301,Abril-82392,Norte,32555
9824,2021-12-27,Brazil,Tocantins,1572866,234113,0,3927,0,12,2021,...,4,4264,423708,Dezembro,Março,Abril,Março-2205301,Abril-82392,Norte,32555
9825,2021-12-28,Brazil,Tocantins,1572866,234964,851,3933,6,12,2021,...,4,4264,423708,Dezembro,Março,Abril,Março-2205301,Abril-82392,Norte,32555
9826,2021-12-29,Brazil,Tocantins,1572866,235340,376,3936,3,12,2021,...,4,4264,423708,Dezembro,Março,Abril,Março-2205301,Abril-82392,Norte,32555
9827,2021-12-30,Brazil,Tocantins,1572866,235558,218,3939,3,12,2021,...,4,4264,423708,Dezembro,Março,Abril,Março-2205301,Abril-82392,Norte,32555


In [47]:
# aqui estamos agrupando a partir do mês e região e somando as mortes
casos_mensais['mortes_regiao_mensal'] = casos_mensais.groupby(['regiao', 'month'])['deaths_1d'].transform('sum')
casos_mensais.tail()

,date,country,state,population,confirmed,confirmed_1d,deaths,deaths_1d,month,year,...,total_mortes_por_mes,mortes_acumulado_mensal,nome_do_mes,nome_mes_mais_casos,nome_mes_mais_mortes,mes_mais_caso,mes_mais_mortes,regiao,casos_regiao_mensal,mortes_regiao_mensal
9823,2021-12-26,Brazil,Tocantins,1572866,234113,0,3927,0,12,2021,...,4264,423708,Dezembro,Março,Abril,Março-2205301,Abril-82392,Norte,32555,370
9824,2021-12-27,Brazil,Tocantins,1572866,234113,0,3927,0,12,2021,...,4264,423708,Dezembro,Março,Abril,Março-2205301,Abril-82392,Norte,32555,370
9825,2021-12-28,Brazil,Tocantins,1572866,234964,851,3933,6,12,2021,...,4264,423708,Dezembro,Março,Abril,Março-2205301,Abril-82392,Norte,32555,370
9826,2021-12-29,Brazil,Tocantins,1572866,235340,376,3936,3,12,2021,...,4264,423708,Dezembro,Março,Abril,Março-2205301,Abril-82392,Norte,32555,370
9827,2021-12-30,Brazil,Tocantins,1572866,235558,218,3939,3,12,2021,...,4264,423708,Dezembro,Março,Abril,Março-2205301,Abril-82392,Norte,32555,370


In [48]:
# Calculando acumulado mensal de casos por região
casos_mensais['casos_regiao_mensal_acumulado'] = casos_mensais.groupby('regiao')['confirmed_1d'].cumsum()

In [49]:
# Calcular acumulado mensal de mortes por região
casos_mensais['mortes_regiao_mensal_acumulado'] = casos_mensais.groupby('regiao')['deaths_1d'].cumsum()

In [50]:
casos_mensais.tail()

,date,country,state,population,confirmed,confirmed_1d,deaths,deaths_1d,month,year,...,nome_do_mes,nome_mes_mais_casos,nome_mes_mais_mortes,mes_mais_caso,mes_mais_mortes,regiao,casos_regiao_mensal,mortes_regiao_mensal,casos_regiao_mensal_acumulado,mortes_regiao_mensal_acumulado
9823,2021-12-26,Brazil,Tocantins,1572866,234113,0,3927,0,12,2021,...,Dezembro,Março,Abril,Março-2205301,Abril-82392,Norte,32555,370,1061003,29460
9824,2021-12-27,Brazil,Tocantins,1572866,234113,0,3927,0,12,2021,...,Dezembro,Março,Abril,Março-2205301,Abril-82392,Norte,32555,370,1061003,29460
9825,2021-12-28,Brazil,Tocantins,1572866,234964,851,3933,6,12,2021,...,Dezembro,Março,Abril,Março-2205301,Abril-82392,Norte,32555,370,1061854,29466
9826,2021-12-29,Brazil,Tocantins,1572866,235340,376,3936,3,12,2021,...,Dezembro,Março,Abril,Março-2205301,Abril-82392,Norte,32555,370,1062230,29469
9827,2021-12-30,Brazil,Tocantins,1572866,235558,218,3939,3,12,2021,...,Dezembro,Março,Abril,Março-2205301,Abril-82392,Norte,32555,370,1062448,29472


In [51]:
casos_mensais.info()

<class 'pandas.core.frame.DataFrame'>RangeIndex: 9828 entries, 0 to 9827Data columns (total 26 columns): #   Column                          Non-Null Count  Dtype         ---  ------                          --------------  -----          0   date                            9828 non-null   datetime64[ns] 1   country                         9828 non-null   object         2   state                           9828 non-null   object         3   population                      9828 non-null   Int64          4   confirmed                       9828 non-null   int64          5   confirmed_1d                    9801 non-null   Int64          6   deaths                          9828 non-null   int64          7   deaths_1d                       9801 non-null   Int64          8   month                           9828 non-null   int32          9   year                            9828 non-null   object         10  mes_com_mais_casos              9828 non-null   int32          11  total_casos_mes     

In [52]:
casos_mensais['mortes_acumulado_mensal'] = casos_mensais['mortes_acumulado_mensal'].astype('Int64')
casos_mensais['year'] = casos_mensais['year'].astype('Int64')

In [53]:
casos_mensais.columns

Index(['date', 'country', 'state', 'population', 'confirmed', 'confirmed_1d',       'deaths', 'deaths_1d', 'month', 'year', 'mes_com_mais_casos',       'total_casos_mes', 'casos_acumulado_mensal', 'mes_com_mais_mortes',       'total_mortes_por_mes', 'mortes_acumulado_mensal', 'nome_do_mes',       'nome_mes_mais_casos', 'nome_mes_mais_mortes', 'mes_mais_caso',       'mes_mais_mortes', 'regiao', 'casos_regiao_mensal',       'mortes_regiao_mensal', 'casos_regiao_mensal_acumulado',       'mortes_regiao_mensal_acumulado'],      dtype='object')

In [54]:
casos_mensais.columns = ['data', 'país', 'estado', 'populacao', 'acumulado_infectados', 'infectados_diario', 'acumulado_mortes', 'mortes_diarias','mes', 'ano', 'mes_com_mais_casos','total_casos_mes', 'casos_acumulado_mensal', 'mes_com_mais_mortes','total_mortes_por_mes', 'mortes_acumulado_mensal','nome_do_mes','nome_mes_mais_casos', 'nome_mes_mais_mortes', 'mes_mais_caso','mes_mais_mortes', 'regiao','casos_regiao_mensal', 'mortes_regiao_mensal','casos_regiao_mensal_acumulado', 'mortes_regiao_mensal_acumulado']

In [55]:
casos_mensais.head(3)

,data,país,estado,populacao,acumulado_infectados,infectados_diario,acumulado_mortes,mortes_diarias,mes,ano,...,nome_do_mes,nome_mes_mais_casos,nome_mes_mais_mortes,mes_mais_caso,mes_mais_mortes,regiao,casos_regiao_mensal,mortes_regiao_mensal,casos_regiao_mensal_acumulado,mortes_regiao_mensal_acumulado
0,2021-01-01,Brazil,Acre,881935,41689,<NA>,796,<NA>,1,2021,...,Janeiro,Março,Abril,Março-2205301,Abril-82392,Norte,162783,4110,<NA>,<NA>
1,2021-01-02,Brazil,Acre,881935,41941,252,798,2,1,2021,...,Janeiro,Março,Abril,Março-2205301,Abril-82392,Norte,162783,4110,252,2
2,2021-01-03,Brazil,Acre,881935,42046,105,802,4,1,2021,...,Janeiro,Março,Abril,Março-2205301,Abril-82392,Norte,162783,4110,357,6


#### **2.1.5. Carregamento Métricas**

Com os dados manipulados, vamos persisti-lo em disco, fazer o seu download e carrega-lo no Google Data Studio.

In [56]:
casos_mensais.to_csv('./casos-mensais-covid2.csv', sep=',', index=False)

### **2.2. Vacinação**

Vamos processar os dados de **vacinação** da universidade de Oxford.

#### **2.2.1. Extração**

Os dados estão compilados em um único arquivo.

In [57]:
vaccines = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv', sep=',')

In [ ]:
vaccines.head()

Vamos selecionar as colunas de interesse e as linhas referentes ao Brasil.

In [58]:
vaccines = vaccines.query('location == "Brazil"').reset_index(drop=True)
vaccines = vaccines[['location', 'population', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'total_boosters', 'date']]

In [60]:
vaccines.head()

,location,population,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,date
0,Brazil,215313504,NaN,NaN,NaN,NaN,2020-01-05
1,Brazil,215313504,NaN,NaN,NaN,NaN,2020-01-06
2,Brazil,215313504,NaN,NaN,NaN,NaN,2020-01-07
3,Brazil,215313504,NaN,NaN,NaN,NaN,2020-01-08
4,Brazil,215313504,NaN,NaN,NaN,NaN,2020-01-09


#### **2.2.2. Wrangling**

Vamos manipular os dados para o dashboard. O foco é em garantir uma boa granularidade e qualidade da base de dados.

In [61]:
vaccines.head()

,location,population,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,date
0,Brazil,215313504,NaN,NaN,NaN,NaN,2020-01-05
1,Brazil,215313504,NaN,NaN,NaN,NaN,2020-01-06
2,Brazil,215313504,NaN,NaN,NaN,NaN,2020-01-07
3,Brazil,215313504,NaN,NaN,NaN,NaN,2020-01-08
4,Brazil,215313504,NaN,NaN,NaN,NaN,2020-01-09


In [62]:
vaccines.shape

(1674, 7)

In [63]:
vaccines.info()

<class 'pandas.core.frame.DataFrame'>RangeIndex: 1674 entries, 0 to 1673Data columns (total 7 columns): #   Column                   Non-Null Count  Dtype  ---  ------                   --------------  -----   0   location                 1674 non-null   object  1   population               1674 non-null   int64   2   total_vaccinations       1296 non-null   float64 3   people_vaccinated        1296 non-null   float64 4   people_fully_vaccinated  1277 non-null   float64 5   total_boosters           1068 non-null   float64 6   date                     1674 non-null   object dtypes: float64(4), int64(1), object(2)memory usage: 91.7+ KB

Vamos começar tratando os dados faltantes, a estratégia será a de preencher os buracos com o valor anterior válido mais próximo.

In [64]:
vaccines = vaccines.ffill()

Vamos também filtrar a base de dados de acordo com a coluna `date` para garantir que ambas as bases de dados tratam do mesmo período de tempo.

In [65]:
vaccines = vaccines[(vaccines['date'] >= '2021-01-01') & (vaccines['date'] <= '2021-12-31')].reset_index(drop=True)

Agora, vamos alterar o nome das colunas.

In [66]:
vaccines = vaccines.rename(
  columns={
    'location': 'country',
    'total_vaccinations': 'total',
    'people_vaccinated': 'one_shot',
    'people_fully_vaccinated': 'two_shots',
    'total_boosters': 'three_shots',
  }
)

Vamos então computar novas colunas para enriquecer a base de dados.

 - Dados relativos:

In [67]:
vaccines['one_shot_perc'] = round(vaccines['one_shot'] / vaccines['population'], 4)
vaccines['two_shots_perc'] = round(vaccines['two_shots'] / vaccines['population'], 4)
vaccines['three_shots_perc'] = round(vaccines['three_shots'] / vaccines['population'], 4)

Garantir o tipo do dado é fundamental para consistência da base de dados. Vamos fazer o *type casting* das colunas.

In [68]:
vaccines['population'] = vaccines['population'].astype('Int64')
vaccines['total'] = vaccines['total'].astype('Int64')
vaccines['one_shot'] = vaccines['one_shot'].astype('Int64')
vaccines['two_shots'] = vaccines['two_shots'].astype('Int64')
vaccines['three_shots'] = vaccines['three_shots'].astype('Int64')

Vamos reorganizar as colunas

In [69]:
vaccines = vaccines[['date', 'country', 'population', 'total', 'one_shot', 'one_shot_perc', 'two_shots', 'two_shots_perc', 'three_shots', 'three_shots_perc']]

Por fim vamos traduzir o nome do DataFrame e as colunas e conferir o resultado

In [70]:
vacinas=vaccines.copy()

In [71]:
vacinas.columns =['data', 'país', 'populacao', 'total_doses_acumulado', 'acumulado_pessoas_com_1_dose', 'percentual_pessoas_1_dose', 'acumulado_pessoas_com_2_doses', 'percentual_pessoas_2_doses', 'acumulado_pessoas_3_doses', 'percentual_pessoas_3_doses']

In [72]:
vacinas.tail()

,data,país,populacao,total_doses_acumulado,acumulado_pessoas_com_1_dose,percentual_pessoas_1_dose,acumulado_pessoas_com_2_doses,percentual_pessoas_2_doses,acumulado_pessoas_3_doses,percentual_pessoas_3_doses
360,2021-12-27,Brazil,215313504,329011365,165952037,0.7707,142764283,0.6631,25218893,0.1171
361,2021-12-28,Brazil,215313504,329861730,166062249,0.7713,142965728,0.6640,25758909,0.1196
362,2021-12-29,Brazil,215313504,330718457,166143380,0.7716,143282084,0.6655,26219623,0.1218
363,2021-12-30,Brazil,215313504,331164041,166185628,0.7718,143398692,0.6660,26507937,0.1231
364,2021-12-31,Brazil,215313504,331273910,166195505,0.7719,143436012,0.6662,26571077,0.1234


#### **2.2.3. Carregamento**

Com os dados manipulados, vamos persisti-lo em disco, fazer o seu download e carrega-lo no Google Data Studio.

In [73]:
vacinas.to_csv('./vacinas-covid2.csv', sep=',', index=False)

# **COVID Dashboard**

## 3\. Exploração Interativa de Dados

### **3.1. KPIs**

O dashboard de dados contem os seguintes indicadores chaves de desempenho (*key performance indicator* ou KPI) consolidados:

1. Mes com mais casos do ano 2021
1. Mês com mais mortes do ano 2021

### **3.2. EDA**

O dashboard de dados contem os seguintes gráficos para a análise exploratória de dados (*exploratory data analysis*
ou EDA) interativa:

1. Distribuição de casos acumulado e não acumulados por mes do ano 2021
1. distribuição de mortes acumulado e não acumulados por mes do ano 2021
1. Distribuição de casos e mortes por região no ano de 2021
1. distribuição de vacinação da primeira ,segunda e terceira dose por mes no ano de 2021